### The RAAS System

In [1]:
import os
import math
import numpy as np
import pathlib

from Bio import Entrez
from Bio import SeqIO
from Bio.Align import PairwiseAligner, substitution_matrices
from Bio.Phylo.TreeConstruction import DistanceMatrix, DistanceTreeConstructor
from Bio.Phylo import draw, draw_ascii

accession_dict = {
    "Human": "NP_001358344.1",
    "Chimpanzee": "NP_001008995.1",
    "Marmoset": "F7CNJ6",
    "Cat": "A0A384DV19",
    "Dog": "J9P7Y2",
    "Ferret": "BAE53380.1",
    "Bat": "ADN93472.1",
    "Pangolin": "XP_017505752.1",
    "Pig": "NP_001116542.1",
    "Mouse": "NP_081562.2",
    "Cattle": "Q58DD0",
}

In [5]:
fasta_file = "ace2_orthologs.fasta"
accession_vals = list(accession_dict.values())

# Recommended practice daw na maglagay ng email or sumn but ok
try:
    handle = Entrez.efetch(db="protein", id=accession_vals, rettype="fasta", retmode="text")
    data = handle.read()
    handle.close()
    pathlib.Path(fasta_file).write_text(data)
    print(f"Saved fasta file to {fasta_file}")
except Exception as e:
    print(f"Errror occured: {e}")

Saved fasta file to ace2_orthologs.fasta


In [13]:
name_to_record_map = {}
try:
    with open(fasta_filename, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            common_name = None
            for name, acc in accession_dict.items():
                if acc in record.id:
                    common_name = name
                    break
            if common_name:
                record.id = common_name
                record.description = f"{common_name} ACE2"
                name_to_record_map[common_name] = record

    print(">>> Successfully loaded sequences from file. Verification:")
    print("---------------------------------------------------------")
    for name, record in name_to_record_map.items():
        print(f"  - {name:<12} ({accession_dict[name]}): {len(record.seq)} amino acids")
    print("---------------------------------------------------------")

except FileNotFoundError:
    print(f"\n!!! Error: The file '{fasta_filename}' was not found. Please run the download step again.")

>>> Successfully loaded sequences from file. Verification:
---------------------------------------------------------
  - Human        (NP_001358344.1): 805 amino acids
  - Chimpanzee   (NP_001008995.1): 1304 amino acids
  - Ferret       (BAE53380.1): 805 amino acids
  - Bat          (ADN93472.1): 805 amino acids
  - Pangolin     (XP_017505752.1): 805 amino acids
  - Pig          (NP_001116542.1): 805 amino acids
  - Mouse        (NP_081562.2): 805 amino acids
  - Cattle       (Q58DD0): 804 amino acids
---------------------------------------------------------


In [15]:
# 4. Pairwise Global Alignment and Distance Calculation
aligner = PairwiseAligner()
aligner.mode = 'global'
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5

def calculate_poisson_distance(aligned_seq1, aligned_seq2):
    """Calculates Poisson distance from two aligned sequences, ignoring gaps."""
    mismatches = 0
    aligned_len_no_gaps = 0
    for i in range(len(aligned_seq1)):
        if aligned_seq1[i] != '-' and aligned_seq2[i] != '-':
            aligned_len_no_gaps += 1
            if aligned_seq1[i] != aligned_seq2[i]:
                mismatches += 1
    
    if aligned_len_no_gaps == 0:
        return 0.0

    # p-distance is the proportion of non-identical sites
    p_dist = mismatches / aligned_len_no_gaps
    
    # Avoid a math domain error if sequences are 100% different
    if p_dist >= 1.0:
        return float('inf')
    # Avoid taking the log of 1 if sequences are identical
    elif p_dist == 0.0:
        return 0.0
    else:
        # Poisson correction formula
        return -math.log(1.0 - p_dist)

species_names = list(name_to_record_map.keys())
num_species = len(species_names)
distance_matrix_data = []

print(f"Computing all-vs-all distance matrix for {num_species} species...")

# Iterate through the species to build the lower-triangular matrix
for i in range(num_species):
    row = []
    for j in range(i):
        species1_name = species_names[i]
        species2_name = species_names[j]
        
        # A print statement to show progress and help debug if needed
        # print(f"  Aligning {species1_name} vs {species2_name}...")

        record1 = name_to_record_map[species1_name]
        record2 = name_to_record_map[species2_name]
        
        # Perform alignment and get the single best alignment
        alignment = aligner.align(record1.seq, record2.seq)[0]
        
        # Calculate distance and add it to the current row
        poisson_dist = calculate_poisson_distance(alignment[0], alignment[1])
        row.append(poisson_dist)
        
    # Append the completed row to the matrix. The length of 'row' will be 'i'.
    distance_matrix_data.append(row)

# This is the line that caused the error. Now it should work correctly.
distance_matrix = DistanceMatrix(names=species_names, matrix=distance_matrix_data)

print("Done.\n")
print("Poisson Corrected Distance Matrix:")
print(distance_matrix)

Computing all-vs-all distance matrix for 8 species...


ValueError: 'matrix' should be in lower triangle format

In [7]:
# --- Step 5: Align Sequences and Analyze Key Residues ---
print(">>> Performing pairwise alignments against Human ACE2 and analyzing key residues...")
print("-" * 70)
print(f"{'Species':<12} | {'Accession':<18} | {'Score':>7} | {'Identity (%)':>12} | {'Pos 41':>7} | {'Pos 42':>7}")
print("-" * 70)

# Store alignments for later use
alignments_dict = {}

# The key positions are 1-based, Python is 0-based.
# Position 41 in biology is index 40 in Python.
pos_41_index = 40
pos_42_index = 41

for accession, record in sequences_dict.items():
    if accession == human_accession:
        continue # Skip aligning human to itself

    # Perform the global alignment
    # We take the first alignment, which is the one with the optimal score.
    alignment = aligner.align(human_record.seq, record.seq)[0]
    alignments_dict[accession] = alignment

    # Calculate percent identity
    # (Number of identical sites) / (Length of alignment)
    identity = (np.array(list(alignment[0])) == np.array(list(alignment[1]))).sum()
    percent_identity = (identity / alignment.shape[1]) * 100

    # Find the corresponding residues at key positions
    # We need to map the original sequence index to the alignment index
    human_aligned_seq = alignment[0]
    other_aligned_seq = alignment[1]
    
    # This finds the index in the alignment that corresponds to the original ungapped index
    map_human_to_alignment = [i for i, char in enumerate(human_aligned_seq) if char != '-']
    
    # Get the characters at the key positions in the alignment
    aligned_idx_41 = map_human_to_alignment[pos_41_index]
    aligned_idx_42 = map_human_to_alignment[pos_42_index]
    
    human_res_41 = human_aligned_seq[aligned_idx_41]
    other_res_41 = other_aligned_seq[aligned_idx_41]
    
    human_res_42 = human_aligned_seq[aligned_idx_42]
    other_res_42 = other_aligned_seq[aligned_idx_42]

    # Get the common name for display
    common_name = [name for name, acc in accession_dict.items() if acc == accession][0]
    
    # Print the summary row
    print(f"{common_name:<12} | {accession:<18} | {alignment.score:7.1f} | {percent_identity:11.2f}% | {other_res_41:^7} | {other_res_42:^7}")

print("-" * 70)
print("\nDiscussion Point: Note the residues for Marmoset (H, E) at positions 41 and 42,")
print("which differ from the Human (Y, Q), as highlighted in the PNAS paper.")
# You can print a full alignment to inspect it
print("\nExample alignment (Human vs. Marmoset):")
print(alignments_dict[accession_dict["Marmoset"]])

>>> Performing pairwise alignments against Human ACE2 and analyzing key residues...
----------------------------------------------------------------------
Species      | Accession          |   Score | Identity (%) |  Pos 41 |  Pos 42
----------------------------------------------------------------------
Chimpanzee   | NP_001008995.1     |  1152.0 |       24.36% |    F    |    Q   
Ferret       | BAE53380.1         |  3628.0 |       82.61% |    Y    |    Q   
Bat          | ADN93472.1         |  3515.0 |       80.12% |    H    |    Q   
Pangolin     | XP_017505752.1     |  3679.0 |       84.84% |    Y    |    Q   
Pig          | NP_001116542.1     |  3567.0 |       81.37% |    Y    |    Q   
Mouse        | NP_081562.2        |  3579.0 |       82.11% |    Y    |    Q   


IndexError: list index out of range